# Suggestive or Non-Suggestive Binary Classifier

In [0]:
!nvidia-smi

Fri Mar 22 17:20:11 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.43       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### Importing Dependencies

In [0]:
import pandas as pd                    

!pip install flair

from flair.data import Sentence

from flair.data_fetcher import NLPTaskDataFetcher       
from flair.embeddings import WordEmbeddings, DocumentRNNEmbeddings       
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

from pathlib import Path


    100% |████████████████████████████████| 81kB 3.1MB/s 
    100% |████████████████████████████████| 798kB 24.1MB/s 
    100% |████████████████████████████████| 122kB 31.8MB/s 
    100% |████████████████████████████████| 1.0MB 21.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/70/55/6b/c12cf58209b8346f653a04f37dd8f607ab0e85a26238a23420
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
Successfully built flair segtok mpld3 sqlitedict
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


## Data Processing

In [0]:
from google.colab import files

file_name = files.upload()

Saving SubtaskA_EvaluationData.csv to SubtaskA_EvaluationData.csv
Saving SubtaskA_Trial_Test_Labeled.csv to SubtaskA_Trial_Test_Labeled.csv
Saving V1.4_Training.csv to V1.4_Training.csv


In [0]:
import os

file_name = list(file_name)[0]
file_name_tmp = file_name.strip().replace(" ", "_")

if file_name != file_name_tmp:
  os.rename(file_name, file_name_tmp)
  file_name = file_name_tmp

print("Filename :", file_name)

Filename : SubtaskA_EvaluationData.csv


### Using Pandas to read `csv` and getting columns 

Training Data

In [0]:
colnames = ["corpus", "label"]
data_train = pd.read_csv("V1.4_Training.csv", usecols=[1,2], names=colnames, header=None)
columnsTitles = ["label", "corpus"]
data_train = data_train.reindex(columns=columnsTitles)
data_train['label'] = '__label__' + data_train['label'].astype(str)


Evaluation Data

In [0]:
colnames = ["corpus", "label"]

data_eva = pd.read_csv("SubtaskA_EvaluationData.csv", usecols=[1,2], names=colnames, header=None)
columnsTitles = ["label", "corpus"]
data_eva = data_eva.reindex(columns=columnsTitles)
data_eva['label'] = '__label__' + data_eva['label'].astype(str)



Test Data

In [0]:
colnames = ["corpus", "label"]
data_tes = pd.read_csv("SubtaskA_Trial_Test_Labeled.csv", usecols=[1,2], names=colnames, header=None,encoding='cp1252')
columnsTitles = ["label", "corpus"]
data_tes = data_tes.reindex(columns=columnsTitles)
data_tes['label'] = '__label__' + data_tes['label'].astype(str)
data_tes.shape

(592, 2)

### Splitting the data to test, train and development set  
$train = 0.8*data$  
$test = 0.1*data$  
$dev = 0.1*data$

In [0]:
data_train.to_csv('train.csv',sep='\t',index = False, header = False)
data_tes.to_csv('test.csv',sep='\t', index = False, header = False)
data_eva.to_csv('dev.csv', sep='\t',index = False, header = False)

## Using $Flair$ framework for classification
$Flair$ is a NLP framework build on top of $pytorch$

In [0]:
corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')

2019-03-22 17:21:38,419 Reading data from .
2019-03-22 17:21:38,422 Train: train.csv
2019-03-22 17:21:38,423 Dev: dev.csv
2019-03-22 17:21:38,425 Test: test.csv


### Word Embedding  
Using $GLoVe$ word embedding 

In [0]:
word_embedding = [WordEmbeddings('glove')]

2019-03-22 17:21:42,241 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpf8lg7lts


100%|██████████| 160000128/160000128 [00:06<00:00, 23517744.97B/s]

2019-03-22 17:21:49,575 copying /tmp/tmpf8lg7lts to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2019-03-22 17:21:49,899 removing temp file /tmp/tmpf8lg7lts
2019-03-22 17:21:50,335 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim not found in cache, downloading to /tmp/tmprkuw1ztq


100%|██████████| 21494764/21494764 [00:01<00:00, 13391383.53B/s]

2019-03-22 17:21:52,539 copying /tmp/tmprkuw1ztq to cache at /root/.flair/embeddings/glove.gensim
2019-03-22 17:21:52,573 removing temp file /tmp/tmprkuw1ztq


### Creating a dictionary of labels

In [0]:
label_dict = corpus.make_label_dictionary()

### Embedding the sentence  
Document embedding are created from embeddings from all the words in sentence.

In [0]:
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embedding)

### Building the classifier

In [0]:
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, multi_label=False)

### Training the classifier

In [0]:
trainer = ModelTrainer(classifier, corpus)

In [0]:
trainer.train('./',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5,
              max_epochs=150)

2019-03-22 17:21:59,522 ----------------------------------------------------------------------------------------------------
2019-03-22 17:21:59,524 Evaluation method: MICRO_F1_SCORE
2019-03-22 17:21:59,526 ----------------------------------------------------------------------------------------------------
2019-03-22 17:21:59,665 epoch 1 - iter 0/266 - loss 0.01868144
2019-03-22 17:22:00,903 epoch 1 - iter 26/266 - loss 0.01881932
2019-03-22 17:22:02,129 epoch 1 - iter 52/266 - loss 0.01834573
2019-03-22 17:22:03,251 epoch 1 - iter 78/266 - loss 0.01819718
2019-03-22 17:22:04,417 epoch 1 - iter 104/266 - loss 0.01800613
2019-03-22 17:22:05,684 epoch 1 - iter 130/266 - loss 0.01797560
2019-03-22 17:22:07,121 epoch 1 - iter 156/266 - loss 0.01773021
2019-03-22 17:22:08,286 epoch 1 - iter 182/266 - loss 0.01782193
2019-03-22 17:22:09,685 epoch 1 - iter 208/266 - loss 0.01773902
2019-03-22 17:22:10,840 epoch 1 - iter 234/266 - loss 0.01778013
2019-03-22 17:22:12,012 epoch 1 - iter 260/266 

### Prediction

Prediction on `SubtaskA_EvaluationData.csv`

In [0]:
file_name = files.upload()

In [0]:
file_name = list(file_name)[0]
file_name_tmp = file_name.strip().replace(" ", "_")

if file_name != file_name_tmp:
  os.rename(file_name, file_name_tmp)
  file_name = file_name_tmp

print("Filename :", file_name)

In [0]:
colnames = ["id","corpus", "label"]

data_un_test = pd.read_csv("SubtaskA_EvaluationData.csv", usecols=[0,1,2], names=colnames, header=None)

In [0]:
test_un_sentences = data_un_test["corpus"].tolist()

In [0]:
correct = 0
total = 0
labels_test = []
for sentence in test_un_sentences:
  sentence = Sentence(sentence)
  classifier.predict(sentence)
  label = str(sentence.labels[0]).split()[0]
  labels_test.append(label)

In [0]:
data_un_test["label"] = labels_test

In [0]:
data_un_test.to_csv("tezuesh_varshney.csv", sep='\t', encoding='utf-8')

In [0]:
files.download('tezuesh_varshney.csv') 